<a href="https://colab.research.google.com/github/okana2ki/gai4e/blob/main/emotion_sns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q -U google-genai gradio wordcloud matplotlib japanize-matplotlib

import gradio as gr
import threading, time
from queue import Queue
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import japanize_matplotlib
import json
import google.genai as genai
from google.colab import userdata
from google.genai import types

# --- API 初期化 ---
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

PRIMARY_MODEL = "gemini-2.5-flash-lite"
FALLBACK_MODEL = "gemini-2.0-flash-lite"

# --- 投稿キューと集計 ---
llm_queue = Queue()
processed_posts = []
processed_count = 0
queue_history = []

# --- キュー可視化 ---
def visualize_queue():
    global llm_queue, processed_count
    return {"Queue": llm_queue.qsize(), "Processed": processed_count}

# --- LLM 呼び出し関数 ---
def call_llm(post_text):
    instruction = (
        "投稿テキストを受け取り、感情分析と重要ワード抽出をJSON形式で返してください。"
        "形式: {\"sentiment\": \"joy\", \"keywords\": [\"word1\",\"word2\"]}"
    )
    models_to_try = [PRIMARY_MODEL, FALLBACK_MODEL]

    for model in models_to_try:
        try:
            response = client.models.generate_content(
                model=model,
                config=types.GenerateContentConfig(system_instruction=instruction),
                contents=post_text
            )
            return json.loads(response.text)
        except Exception as e:
            print(f"モデル {model} でエラー: {e}")
            continue

    # どちらも失敗
    return {"sentiment": "error", "keywords": []}

# --- ワーカースレッド: 4秒ごとに1件処理 ---
def llm_worker():
    global processed_count, processed_posts
    while True:
        if not llm_queue.empty():
            post = llm_queue.get()
            result = call_llm(post)
            processed_posts.append(result)
            processed_count += 1
        time.sleep(4)  # 15RPM 相当

threading.Thread(target=llm_worker, daemon=True).start()

# --- 集計グラフ作成 ---
def generate_graphs():
    # 感情集計
    sentiments = [p["sentiment"] for p in processed_posts if p["sentiment"] != "error"]
    sentiment_labels = ["joy", "sadness", "anger", "fear", "neutral", "surprise"]
    counts = [sentiments.count(label) for label in sentiment_labels]

    # 棒グラフ
    fig, ax = plt.subplots(1,2, figsize=(12,4))
    ax[0].bar(sentiment_labels, counts, color='skyblue')
    ax[0].set_title("感情分布")

    # ワードクラウド
    all_words = []
    for p in processed_posts:
        all_words.extend(p.get("keywords", []))
    if all_words:
        wc_text = " ".join(all_words)
        wc = WordCloud(font_path="/usr/share/fonts/truetype/NotoSansJP-Regular.otf",
                       width=600, height=400, background_color="white").generate(wc_text)
        ax[1].imshow(wc, interpolation="bilinear")
        ax[1].axis("off")
        ax[1].set_title("重要ワード")
    else:
        ax[1].text(0.5,0.5,"ワードなし", ha="center")
        ax[1].axis("off")

    plt.tight_layout()
    return fig

# --- 投稿関数 ---
def submit_post(post_text):
    llm_queue.put(post_text)
    return generate_graphs(), visualize_queue()

# --- 表示更新ボタン ---
def refresh_display():
    return generate_graphs(), visualize_queue()

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("## 高校生向け Gemini API レート制限デモ")
    with gr.Row():
        input_text = gr.Textbox(label="投稿内容")
        submit_btn = gr.Button("投稿")
        refresh_btn = gr.Button("表示更新")

    output_graph = gr.Plot()
    queue_status = gr.Label()

    submit_btn.click(fn=submit_post, inputs=input_text, outputs=[output_graph, queue_status])
    refresh_btn.click(fn=refresh_display, inputs=None, outputs=[output_graph, queue_status])

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 39.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed55ec6b77d320c6c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
import gradio as gr
import time
import threading
from queue import Queue
import google.genai as genai
from google.colab import userdata
from google.genai import types

# APIキーとクライアント初期化
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

# モデルとフォールバック
PRIMARY_MODEL = "gemini-2.5-flash-lite"
FALLBACK_MODEL = "gemini-2.0-flash-lite"

# 投稿キュー
llm_queue = Queue()
processed_count = 0
queue_history = []

# キュー可視化関数
def visualize_queue():
    global llm_queue, processed_count
    return {"Queue": llm_queue.qsize(), "Processed": processed_count}

# LLM 呼び出し関数（フォールバック対応）
def call_llm(post_text):
    global client
    instruction = (
        "投稿テキストを受け取り、感情分析と重要ワード抽出をJSON形式で返してください。"
        "形式: {\"sentiment\": \"joy\", \"keywords\": [\"word1\",\"word2\"]}"
    )

    models_to_try = [PRIMARY_MODEL, FALLBACK_MODEL]

    for model in models_to_try:
        try:
            response = client.models.generate_content(
                model=model,
                config=types.GenerateContentConfig(
                    system_instruction=instruction
                ),
                contents=post_text
            )
            return response.text
        except Exception as e:
            print(f"モデル {model} でエラー: {e}")
            continue

    # どちらのモデルでも失敗した場合
    return '{"sentiment": "error", "keywords": []}'

# ワーカースレッド: 4秒ごとに1件処理
def llm_worker():
    global processed_count
    while True:
        if not llm_queue.empty():
            post = llm_queue.get()
            print(f"Processing: {post}")
            result = call_llm(post)  # 実際にAPI呼び出し
            print(f"Result: {result}")
            processed_count += 1
        time.sleep(4)  # 15RPM相当

threading.Thread(target=llm_worker, daemon=True).start()

# 投稿ボタン
def submit_post(post_text):
    llm_queue.put(post_text)
    return visualize_queue()

# UI構築
with gr.Blocks() as demo:
    gr.Markdown("## 高校生向け Gemini API レート制限デモ")
    with gr.Row():
        input_text = gr.Textbox(label="投稿内容")
        submit_btn = gr.Button("投稿")
        refresh_btn = gr.Button("表示更新")

    queue_graph = gr.BarPlot(label="キューと処理状況", x=["Queue","Processed"], y=[0,0], color="blue")

    submit_btn.click(fn=submit_post, inputs=input_text, outputs=queue_graph)
    refresh_btn.click(fn=visualize_queue, inputs=None, outputs=queue_graph)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3913e5a2c6f320b109.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re

# 日本語フォントパス
FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# 投稿用DataFrame
df = pd.DataFrame(columns=["投稿", "sentiment", "ai_comment"])

# --- LLM呼び出し＆JSON抽出 ---
def analyze_post(text):
    global df

    system_instruction = (
        "あなたはSNS分析の専門家です。"
        "以下の文章を解析して、JSON形式で出力してください。"
        "JSONには2つのキーを含めます："
        "  1) 'sentiment' : joy, surprise, confusion, neutral のいずれかで文章の感情を分類してください。"
        "  2) 'ai_comment' : 文章に対して短いポジティブなコメントを生成してください。"
        "出力は必ずJSON形式のみで返してください。"
    )

    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
        contents=text
    )

    # デバッグ用：生レスポンス
    print("Raw LLM response:", response.text)

    # JSON部分だけ抽出
    match = re.search(r"\{.*\}", response.text, re.DOTALL)
    if match:
        try:
            result = json.loads(match.group())
            sentiment = result.get("sentiment", "neutral").strip().lower()
            ai_comment = result.get("ai_comment", "")
        except:
            sentiment = "neutral"
            ai_comment = response.text
    else:
        sentiment = "neutral"
        ai_comment = response.text

    if sentiment not in ["joy","surprise","confusion","neutral"]:
        sentiment = "neutral"

    # 新しい投稿を追加
    df.loc[len(df)] = [text, sentiment, ai_comment]

    # 集計更新
    fig, wc_image = update_visualizations()
    return fig, wc_image, ai_comment

# --- 集計更新関数 ---
def update_visualizations():
    global df

    # 棒グラフ
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["joy","surprise","confusion","neutral"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={
            "joy":"gold",
            "surprise":"lightblue",
            "confusion":"orange",
            "neutral":"lightgrey"
        }
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")

    # ワードクラウド
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()  # PIL.Image形式

    return fig, wc_image

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    gr.Markdown("投稿ごとにAIが感情分析を行い、グラフとワードクラウドが即座に更新されます。")

    input_text = gr.Textbox(label="あなたの感想を入力してください", placeholder="例：楽しかった！")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("集計更新")

    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")

    # 投稿ボタン → 新規投稿＋自動更新
    submit_btn.click(
        fn=analyze_post,
        inputs=input_text,
        outputs=[graph_output, wc_output, ai_comment_output]
    )

    # 更新ボタン → DataFrameは変更せず集計更新
    update_btn.click(
        fn=update_visualizations,
        inputs=None,
        outputs=[graph_output, wc_output]
    )

demo.launch()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://61f4df578d3466e75d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json

# 日本語フォントパス
FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# 投稿用DataFrame
df = pd.DataFrame(columns=["投稿", "sentiment", "ai_comment"])

# --- 1回のLLM呼び出しでJSON取得 ---
def analyze_post(text):
    global df

    system_instruction = (
        "あなたはSNS分析の専門家です。"
        "以下の文章を解析して、JSON形式で出力してください。"
        "JSONには2つのキーを含めます："
        "  1) 'sentiment' : joy, surprise, confusion, neutral のいずれかで文章の感情を分類してください。"
        "  2) 'ai_comment' : 文章に対して短いポジティブなコメントを生成してください。"
        "出力は必ずJSON形式のみで返してください。"
    )

    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
        contents=text
    )

    try:
        result = json.loads(response.text)
        sentiment = result.get("sentiment", "neutral")
        ai_comment = result.get("ai_comment", "")
    except:
        sentiment = "neutral"
        ai_comment = response.text

    if sentiment not in ["joy", "surprise", "confusion", "neutral"]:
        sentiment = "neutral"

    df.loc[len(df)] = [text, sentiment, ai_comment]

    # 集計更新
    fig, wc_image = update_visualizations()
    return fig, wc_image, ai_comment

# --- 集計更新 ---
def update_visualizations():
    global df

    # 棒グラフ
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["joy","surprise","confusion","neutral"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={
            "joy":"gold",
            "surprise":"lightblue",
            "confusion":"orange",
            "neutral":"lightgrey"
        }
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")

    # ワードクラウド
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()  # PIL.Image形式

    return fig, wc_image

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ（リアルタイム自動更新）")
    gr.Markdown("投稿ごとにAIが感情分析を行い、グラフとワードクラウドが即座に更新されます。")

    input_text = gr.Textbox(label="あなたの感想を入力してください", placeholder="例：楽しかった！")
    submit_btn = gr.Button("投稿")

    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")

    # 投稿→解析＋自動更新（出力をflatten）
    submit_btn.click(
        fn=analyze_post,
        inputs=input_text,
        outputs=[graph_output, wc_output, ai_comment_output]
    )

demo.launch()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  fonts-noto-cjk-extra
The following NEW packages will be installed:
  fonts-noto-cjk
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 61.2 MB of archives.
After this operation, 93.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-noto-cjk all 1:20220127+repack1-1 [61.2 MB]
Fetched 61.2 MB in 1s (52.1 MB/s)
Selecting previously unselected package fonts-noto-cjk.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../fonts-noto-cjk_1%3a20220127+repack1-1_all.deb ...
Unpacking fonts-noto-cjk (1:20220127+repack1-1) ...
Setting up fonts-noto-cjk (1:20220127+repack1-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share

In [3]:
# Colab向け
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json
import requests
from io import BytesIO

# --- 日本語フォントを用意（Colab用） ---
!wget -O /usr/share/fonts/truetype/NotoSansJP-Regular.otf https://noto-website-2.storage.googleapis.com/pkgs/NotoSansJP-unhinted.zip
!unzip -o /usr/share/fonts/truetype/NotoSansJP-Regular.otf -d /usr/share/fonts/truetype/
FONT_PATH = "/usr/share/fonts/truetype/NotoSansJP-Regular.otf"

# --- 投稿用DataFrame ---
df = pd.DataFrame(columns=["投稿", "sentiment", "ai_comment"])

# --- 1回のLLM呼び出しでJSON取得 ---
def analyze_post(text):
    global df

    system_instruction = (
        "あなたはSNS分析の専門家です。"
        "以下の文章を解析して、JSON形式で出力してください。"
        "JSONには2つのキーを含めます："
        "  1) 'sentiment' : joy, surprise, confusion, neutral のいずれかで文章の感情を分類してください。"
        "  2) 'ai_comment' : 文章に対して短いポジティブなコメントを生成してください。"
        "出力は必ずJSON形式のみで返してください。"
    )

    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
        contents=text
    )

    try:
        result = json.loads(response.text)
        sentiment = result.get("sentiment", "neutral")
        ai_comment = result.get("ai_comment", "")
    except:
        sentiment = "neutral"
        ai_comment = response.text

    # 感情ラベルを統一
    if sentiment not in ["joy", "surprise", "confusion", "neutral"]:
        sentiment = "neutral"

    # DataFrameに追加
    df.loc[len(df)] = [text, sentiment, ai_comment]

    # 投稿ごとに即座に集計更新
    return update_visualizations(), ai_comment

# --- 集計結果更新 ---
def update_visualizations():
    global df

    # 棒グラフ（ラベル順序固定）
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["joy","surprise","confusion","neutral"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={
            "joy":"gold",
            "surprise":"lightblue",
            "confusion":"orange",
            "neutral":"lightgrey"
        }
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")

    # 日本語対応ワードクラウド
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()  # PIL.Image形式に変換

    return fig, wc_image

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ（リアルタイム自動更新）")
    gr.Markdown("投稿ごとにAIが感情分析を行い、グラフとワードクラウドが即座に更新されます。")

    input_text = gr.Textbox(label="あなたの感想を入力してください", placeholder="例：楽しかった！")
    submit_btn = gr.Button("投稿")

    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")

    # 投稿→解析＋自動更新
    submit_btn.click(
        fn=analyze_post,
        inputs=input_text,
        outputs=[(graph_output, wc_output), ai_comment_output]
    )

demo.launch()


--2025-08-17 14:54:13--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSansJP-unhinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 108.177.121.207, 142.251.189.207, 142.250.125.207, ...
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|108.177.121.207|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2025-08-17 14:54:13 ERROR 403: Forbidden.

Archive:  /usr/share/fonts/truetype/NotoSansJP-Regular.otf
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /usr/share/fonts/truetype/NotoSansJP-Regular.otf or
        /usr/share/fonts/truetype/NotoSansJP-Regular.otf.zip, and cannot find /usr/share/fonts/truetype/NotoSansJP-R

AttributeError: 'tuple' object has no attribute '_id'

In [2]:
# Colab向け
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO
import json

# --- グローバル変数: 投稿を蓄積するDataFrame ---
df = pd.DataFrame(columns=["投稿", "sentiment", "ai_comment"])

# --- LLMを1回だけ呼んで感情とAIコメントをJSONで取得 ---
def analyze_post(text):
    global df

    system_instruction = (
        "あなたはSNS分析の専門家です。"
        "以下の文章を解析して、JSON形式で出力してください。"
        "JSONには2つのキーを含めます："
        "  1) 'sentiment' : joy, surprise, confusion, neutral のいずれかで文章の感情を分類してください。"
        "  2) 'ai_comment' : 文章に対して短いポジティブなコメントを生成してください。"
        "出力は必ずJSON形式のみで返してください。"
    )

    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
        contents=text
    )

    try:
        result = json.loads(response.text)
        sentiment = result.get("sentiment", "neutral")
        ai_comment = result.get("ai_comment", "")
    except:
        sentiment = "neutral"
        ai_comment = response.text

    # DataFrameに追加
    df.loc[len(df)] = [text, sentiment, ai_comment]

    return ai_comment  # AIコメントのみ返す

# --- 集計結果の更新: 棒グラフとワードクラウド ---
def update_visualizations():
    global df

    # 感情分布グラフ
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={
            "joy":"gold",
            "surprise":"lightblue",
            "confusion":"orange",
            "neutral":"lightgrey"
        }
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")

    # ワードクラウド生成
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white").generate(all_text)
    wc_image = wc.to_image()  # PIL.Image形式に変換

    return fig, wc_image

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ（リアルタイム更新）")
    gr.Markdown("投稿ごとにAIが感情分析を行い、最新集計を可視化します。")

    input_text = gr.Textbox(label="あなたの感想を入力してください", placeholder="例：楽しかった！")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("表示更新")

    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")

    # 投稿→LLM解析
    submit_btn.click(analyze_post, inputs=input_text, outputs=ai_comment_output)

    # 表示更新→最新集計描画
    update_btn.click(update_visualizations, outputs=[graph_output, wc_output])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cbf898d7bfdbdeb6c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import json

# データ保存用
df = pd.DataFrame(columns=["投稿", "感情", "AIコメント"])

def analyze_and_comment_single_call(text):
    global df

    # 1回の呼び出しで感情とAIコメントをJSONで返す指示
    system_instruction = (
        "あなたはSNS分析の専門家です。"
        "以下の文章を解析して、JSON形式で出力してください。"
        "JSONには2つのキーを含めます："
        "  1) 'sentiment' : joy, surprise, confusion, neutral のいずれかで文章の感情を分類してください。"
        "  2) 'ai_comment' : 文章に対して短いポジティブなコメントを生成してください。"
        "出力は必ずJSON形式のみで返してください。"
    )

    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
        contents=text
    )

    # GeminiからのテキストをJSONとしてパース
    try:
        result = json.loads(response.text)
        sentiment = result.get("sentiment", "neutral")
        ai_comment = result.get("ai_comment", "")
    except Exception as e:
        # パース失敗時はデフォルト
        sentiment = "neutral"
        ai_comment = response.text

    # データ保存
    df.loc[len(df)] = [text, sentiment, ai_comment]

    # 感情分布グラフ
    fig = px.histogram(df, x="感情", color="感情", text_auto=True,
                       title="オープンキャンパス感想のリアルタイム感情分析",
                       color_discrete_map={
                           "joy":"gold",
                           "surprise":"lightblue",
                           "confusion":"orange",
                           "neutral":"lightgrey"
                       })
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")

    # ワードクラウド生成
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white").generate(all_text)
    plt.figure(figsize=(6,4))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    buf = BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    img_data = base64.b64encode(buf.read()).decode("utf-8")
    buf.close()

    return fig, f"AIコメント: {ai_comment}", f"ワードクラウド", img_data

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNS（1回呼び出し版）")
    gr.Markdown("投稿するとAIが感情を分析してJSON形式で返し、可視化します。")

    input_text = gr.Textbox(label="あなたの感想を入力してください", placeholder="例：楽しかった！")
    graph_output = gr.Plot(label="感情分析グラフ")
    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    wc_output = gr.Image(label="ワードクラウド")

    input_text.submit(analyze_and_comment_single_call,
                      inputs=input_text,
                      outputs=[graph_output, ai_comment_output, wc_output])

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 39.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
It looks like you are running Gradio on a hosted Jupyter 